In [1]:
import numpy as np
import math
import decimal

In [4]:
# f (x,y,z,w) = 12x + 16y + 15z + 20w
# subject to :
#  x - y + 2w ≤ 30
#  6y + 5z -w ≤ 120
#  5y + z ≤ 15
#  3x + 2y + w ≤ 80
#  x,y,z,w ≥ 0

In [8]:
#initia dictionary:

N_indexes = [1,2,3,4]  #non basis variables, indexing by 1 not 0
 
B_indexes = [5,6,7,8]  #basis variables, indexing by 1 not 0

b = np.array([30,120,15,80])  #the b vector

cn = np.array([12,16,15,20])  #the c vector (objective function coefficients)

A = np.array([[1,-1,0,2,1,0,0,0], [0,6,5,-1,0,1,0,0], [0,5,1,0,0,0,1,0], [3,2,0,1,0,0,0,1]]) #the A matrix


def simplex(A, b, cn, B_indexes, N_indexes):
    print("A", A, sep="\n")
    if(all(b>0)): #avoiding the 0 value when calculating ratio
        xb = b
        zn = (-1)*cn
        #print("simplex simple")
        #any(i<=0 for i in zn)
        iteration=1
        while(any(zn<0)): #zn contains some negative values
            print("------------iteration:{}-------------".format(iteration))
            print("B indexes",B_indexes)
            print("N indexes", N_indexes)
            B = A[:, [i-1 for i in B_indexes]]
            N = A[:, [i-1 for i in N_indexes]]
            print("N", N, sep="\n")
            print("B", B, sep="\n")
            
            #preparing invB*N
            invB_N = np.dot(np.linalg.inv(B), N)
            print("invB_N",invB_N,sep="\n")
            print("xb", xb)
            print("zn", zn)
            
            #
            print("----------------------------------")
            sols = []
            summ = 0
            rslt = "Z ="
            for i in range(len(cn)):
                try:
                    addto=xb[B_indexes.index(i+1)]
                except ValueError:
                    addto = 0
                summ = summ+addto*cn[i]
                sols.append(addto)
                if(i==0):
                    rslt = rslt+" {}(X{}={})".format(cn[i], i+1, addto)
                else:
                    rslt = rslt+" + {}(X{}={})".format(cn[i], i+1,addto)
            rslt = rslt+" = {}".format(summ)
            print(rslt)
            print("----------------------------------")
            
            #

            #the index in N of entering variable is the index of biggest coeff 
            #in cn, which the same as minimum in zn
            min_index = np.where(zn == zn.min())[0][0] #(xj-th)pay attention to indexing(j+1)

            #the column of this index in invB_N
            delta_xb = invB_N[:, min_index]
            print("delta_xb",delta_xb, ": column in invB_N corr to min var in zn")
            #maximum method to avoic constraint on aik being >0
            ratios = delta_xb/xb
            print("ratios", ratios, ": ratios=delta_xb/xb")
            max_ratio = ratios.max()
            print("max_ratio", max_ratio)
            if(max_ratio<=0):
                print("this problem is unbounded")
                return(0)
            max_index = np.where(ratios == max_ratio)[0][0]
            print("max_index", max_index, "o-index")
            p_ratio = math.pow(max_ratio, -1)
            print("p_ratio", p_ratio, "(max_ratio^-1), to go back like if we used min-ratio method")
            
            #changes in objective function <==> changes in non basic vars in dual
            delta_zn = (-1)*invB_N.transpose()[:, max_index]
            print("delta_zn", delta_zn, ": (-1)*row in invB_N corr to max_index")
            d_ratio = zn[min_index]/delta_zn[min_index]
            print("d_ratio", d_ratio, "d_ratios=min(zn)/min(delta_zn)")

            xb = xb - p_ratio*delta_xb
            print("new xb", xb, "    (xb = xb - p_ratio*delta_xb)")
            print("0 is the leaving variable")
            xb[max_index] = p_ratio
            print("new xb2", xb)

            zn = zn - d_ratio*delta_zn
            print("new zn", zn, "    (zn = zn - d_ratio*delta_zn)")
            print("0 is the entering variable")
            zn[min_index] = d_ratio
            print("new zn2", zn)

            lvar = B_indexes[max_index]
            evar = N_indexes[min_index]
            N_indexes[min_index] = lvar
            B_indexes[max_index] = evar
            
            print("---------------end------------------")
            iteration=iteration+1
            
        print("---------------The end(final dict)--------------")
        print("B indexes",B_indexes)
        print("N indexes", N_indexes)
        B = A[:, [i-1 for i in B_indexes]]
        N = A[:, [i-1 for i in N_indexes]]
        print("N", N, sep="\n")
        print("B", B, sep="\n")
        #
        invB_N = np.dot(np.linalg.inv(B), N)
        print("invB_N",invB_N,sep="\n")
        #
        print("xb", xb)
        print("zn", zn)
        sols = []
        summ = 0
        rslt = "Z ="
        for i in range(len(cn)):
            try:
                addto=xb[B_indexes.index(i+1)]
            except ValueError:
                addto = 0
            summ = summ+addto*cn[i]
            sols.append(addto)
            if(i==0):
                rslt = rslt+" {}(X{}={})".format(cn[i], i+1, addto)
            else:
                rslt = rslt+" + {}(X{}={})".format(cn[i], i+1,addto)
        rslt = rslt+" = {}".format(summ)
        print("----------------------------------")
        print(rslt)
        return(sols)

In [7]:
solution = simplex(A, b, cn, B_indexes, N_indexes)

A
[[ 1 -1  0  2  1  0  0  0]
 [ 0  6  5 -1  0  1  0  0]
 [ 0  5  1  0  0  0  1  0]
 [ 3  2  0  1  0  0  0  1]]
------------iteration:1-------------
B indexes [5, 6, 7, 8]
N indexes [1, 2, 3, 4]
N
[[ 1 -1  0  2]
 [ 0  6  5 -1]
 [ 0  5  1  0]
 [ 3  2  0  1]]
B
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
invB_N
[[ 1. -1.  0.  2.]
 [ 0.  6.  5. -1.]
 [ 0.  5.  1.  0.]
 [ 3.  2.  0.  1.]]
xb [ 30 120  15  80]
zn [-12 -16 -15 -20]
----------------------------------
Z = 12(X1=0) + 16(X2=0) + 15(X3=0) + 20(X4=0) = 0
----------------------------------
delta_xb [ 2. -1.  0.  1.] : column in invB_N corr to min var in zn
ratios [ 0.06666667 -0.00833333  0.          0.0125    ] : ratios=delta_xb/xb
max_ratio 0.06666666666666667
max_index 0 o-index
p_ratio 15.0 (max_ratio^-1), to go back like if we used min-ratio method
delta_zn [-1.  1. -0. -2.] : (-1)*row in invB_N corr to max_index
d_ratio 10.0 d_ratios=min(zn)/min(delta_zn)
new xb [  0. 135.  15.  65.]     (xb = xb - p_ratio*delta_xb)
0 is the 